In [1]:
import matplotlib.pyplot as plt
import torch
import time
import os
os.chdir('D:/test')
from CL_BRUNO_TMLR import *
import pandas as pd
from torch.utils.data import Subset
import torchvision
import torchvision.transforms as transforms
device='cuda'
torch.manual_seed(314159)s
np.random.seed(314159)

In [2]:
preprocess_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5070, 0.4865, 0.4409], std=[0.2673, 0.2564, 0.2762])
])

preprocess_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5070, 0.4865, 0.4409], std=[0.2673, 0.2564, 0.2762])
])


cifar_train = torchvision.datasets.CIFAR100(root='./CIFAR100', train=True, transform=preprocess_train)
cifar_test = torchvision.datasets.CIFAR100(root='./CIFAR100', train=False, transform=preprocess_test)

In [3]:
# pretrain feature vector using the first batch of 10 classes
cifar_01_train = get_subset(list(range(10)), cifar_train)
cifar_01_test = get_subset(list(range(10)), cifar_test)
cifar_01_train_loader = DataLoader(cifar_01_train, batch_size=64)
cifar_01_test_loader = DataLoader(cifar_01_test, batch_size=64)
dataset_sizes = {'train': len(cifar_01_train), 'val': len(cifar_01_test)}
my_loader = {'train': cifar_01_train_loader, 'val': cifar_01_test_loader}

In [4]:
# initialise the model
test_model = CLBruno(x_dim=512, y_dim=128, task_dim=1, cond_dim=129, conv=False, task_num=1,
                     y_cat_num=[10], single_task=True, n_dense_block=6, n_hidden_dense=128,
                     activation=nn.Tanh(), mu_init=0., var_init=1., corr_init=0.1, extractor=True, init_out=10, device=device)
test_model = test_model.to(device)
test_model.init_extractor(my_loader, 30)

Epoch 0/29
Epoch 1/29
Epoch 2/29
Epoch 3/29
Epoch 4/29
Epoch 5/29
Epoch 6/29
Epoch 7/29
Epoch 8/29
Epoch 9/29
Epoch 10/29
Epoch 11/29
Epoch 12/29
Epoch 13/29
Epoch 14/29
Epoch 15/29
Epoch 16/29
Epoch 17/29
Epoch 18/29
Epoch 19/29
Epoch 20/29
Epoch 21/29
Epoch 22/29
Epoch 23/29
Epoch 24/29
Epoch 25/29
Epoch 26/29
Epoch 27/29
Epoch 28/29
Epoch 29/29


In [4]:
cifar_loader_train = DataLoader(cifar_train, batch_size=64)
cifar_loader_test = DataLoader(cifar_test, batch_size=64)

# with torch.no_grad():
#     transformed_x_train = []
#     transformed_y_train = torch.tensor([])
#     for x, y in cifar_loader_train:
#         transformed_x_train += [test_model.my_extractor(x.to(device))]
#         transformed_y_train = torch.cat((transformed_y_train, y))
#     transformed_x_train = torch.vstack(transformed_x_train).cpu()
#     transformed_y_train = transformed_y_train.to(torch.long)

#     transformed_x_test = []
#     transformed_y_test = torch.tensor([])
#     for x, y in cifar_loader_test:
#         transformed_x_test += [test_model.my_extractor(x.to(device))]
#         transformed_y_test = torch.cat((transformed_y_test, y))
#     transformed_x_test = torch.vstack(transformed_x_test).cpu()
#     transformed_y_test = transformed_y_test.to(torch.long)

# pd.DataFrame(transformed_x_train.numpy()).to_csv('iCIFAR100_pretrained_feature_train.csv', index=False)
# pd.DataFrame(transformed_x_test.numpy()).to_csv('iCIFAR100_pretrained_feature_test.csv', index=False)

transformed_x_train = torch.tensor(pd.read_csv('iCIFAR100_pretrained_feature_train.csv').to_numpy(), dtype=torch.float)
transformed_x_test = torch.tensor(pd.read_csv('iCIFAR100_pretrained_feature_test.csv').to_numpy(), dtype=torch.float)

transformed_y_test = torch.tensor(cifar_test.targets, dtype=torch.long)
transformed_y_train = torch.tensor(cifar_train.targets, dtype=torch.long)

# initialise the model
test_model = CLBruno(x_dim=512, y_dim=128, task_dim=1, cond_dim=129, conv=False, task_num=1,
                     y_cat_num=[10], single_task=True, n_dense_block=6, n_hidden_dense=128,
                     activation=nn.Tanh(), mu_init=0., var_init=1., corr_init=0.1, extractor=True, init_out=10, device=device)
test_model = test_model.to(device)

In [5]:
task_split = [list(range(i*10, (i+1)*10)) for i in range(10)]
CIL_cifar_train = {}
CIL_cifar_test = {}
# nhwc to nchw
for _ in range(len(task_split)):
    task_id = np.array([i for i,j in enumerate(transformed_y_train) if j in task_split[_]])
    CIL_cifar_train['X_{}'.format(_)] = transformed_x_train[task_id].to(device)
    CIL_cifar_train['y_{}'.format(_)] = transformed_y_train[task_id].to(device)
    task_id = np.array([i for i, j in enumerate(transformed_y_test) if j in task_split[_]])
    CIL_cifar_test['X_{}'.format(_)] = transformed_x_test[task_id].to(device)
    CIL_cifar_test['y_{}'.format(_)] = transformed_y_test[task_id].to(device)

# CL-BRUNO with functional regularization

In [59]:
# set alignment_reg=0. to turn off alignment regularizer
train_loss, test_loss = test_model.train_init(CIL_cifar_train['X_0'], CIL_cifar_train['y_0'],
                                              torch.zeros(CIL_cifar_train['y_0'].shape[0], dtype=torch.long).to(device),
                                              batch_size=128, epoch=30, weight_decay=0., lr=1e-3, embedding_reg=0.1)
                                              # context_portion=0.2)


N = len(CIL_cifar_test['y_0'])
my_id_test = range(len(CIL_cifar_test['y_0']))
# runnable, check outputs
q = torch.zeros((N, 10))
p = torch.zeros(N)
for i,j in enumerate(my_id_test):
    a, b = test_model.prediction(CIL_cifar_test['X_0'][j], 0)
    q[i] = a.cpu()
    p[i] = b.cpu()
print(torch.sum(q.cpu().argmax(1) != CIL_cifar_test['y_0'][my_id_test].cpu())/N)

tensor(0.1540)


In [2]:
batch_sizes = [128]*9
# doing CIL
for batch_id in range(1, 10):
    train_loss1, test_loss1, reg_loss1 = test_model.train_continual_task(X_new=CIL_cifar_train['X_{}'.format(batch_id)],
                                                                         y_new=CIL_cifar_train['y_{}'.format(batch_id)],
                                                                         task_id=0, epoch=30, batch_size=int(batch_sizes[batch_id-1]),
                                                                         weight_decay=0., lr=1e-3, n_pseudo=128,
                                                                         embedding_reg=0.1)

    acc = 0.
    for hist_id in range(batch_id+1):
        N = len(CIL_cifar_test['y_{}'.format(hist_id)])
        my_id_test = range(len(CIL_cifar_test['y_{}'.format(hist_id)]))
        q = torch.zeros((N, (batch_id + 1) * 10))
        p = torch.zeros(N)
        for i, j in enumerate(my_id_test):
            a, b = test_model.prediction(CIL_cifar_test['X_{}'.format(hist_id)][j], 0)
            q[i] = a.cpu()
            p[i] = b.cpu()
        acc += torch.sum(q.cpu().argmax(1) != CIL_cifar_test['y_{}'.format(hist_id)][my_id_test].cpu()) / N
    print('Batch {}'.format(batch_id))
    print('misclassification: {}'.format(np.round(acc.numpy()/(batch_id+1), 3)))


Batch 1
misclassification: 0.368
Batch 2
misclassification: 0.457
Batch 3
misclassification: 0.522
Batch 4
misclassification: 0.631
Batch 5
misclassification: 0.672
Batch 6
misclassification: 0.708
Batch 7
misclassification: 0.74
Batch 8
misclassification: 0.762
Batch 9
misclassification: 0.788


# working out ECE

In [44]:
# working out ECE, code taken from https://github.com/Jonathan-Pearce/calibration-toolbox/blob/master/metrics.py
from scipy.special import softmax
class CELoss(object):

    def compute_bin_boundaries(self, probabilities = np.array([])):

        #uniform bin spacing
        if probabilities.size == 0:
            bin_boundaries = np.linspace(0, 1, self.n_bins + 1)
            self.bin_lowers = bin_boundaries[:-1]
            self.bin_uppers = bin_boundaries[1:]
        else:
            #size of bins 
            bin_n = int(self.n_data/self.n_bins)

            bin_boundaries = np.array([])

            probabilities_sort = np.sort(probabilities)  

            for i in range(0,self.n_bins):
                bin_boundaries = np.append(bin_boundaries,probabilities_sort[i*bin_n])
            bin_boundaries = np.append(bin_boundaries,1.0)

            self.bin_lowers = bin_boundaries[:-1]
            self.bin_uppers = bin_boundaries[1:]


    def get_probabilities(self, output, labels, logits):
        #If not probabilities apply softmax!
        if logits:
            self.probabilities = softmax(output, axis=1)
        else:
            self.probabilities = output

        self.labels = labels
        self.confidences = np.max(self.probabilities, axis=1)
        self.predictions = np.argmax(self.probabilities, axis=1)
        self.accuracies = np.equal(self.predictions,labels)

    def binary_matrices(self):
        idx = np.arange(self.n_data)
        #make matrices of zeros
        pred_matrix = np.zeros([self.n_data,self.n_class])
        label_matrix = np.zeros([self.n_data,self.n_class])
        #self.acc_matrix = np.zeros([self.n_data,self.n_class])
        pred_matrix[idx,self.predictions] = 1
        label_matrix[idx,self.labels] = 1

        self.acc_matrix = np.equal(pred_matrix, label_matrix)


    def compute_bins(self, index = None):
        self.bin_prop = np.zeros(self.n_bins)
        self.bin_acc = np.zeros(self.n_bins)
        self.bin_conf = np.zeros(self.n_bins)
        self.bin_score = np.zeros(self.n_bins)

        if index == None:
            confidences = self.confidences
            accuracies = self.accuracies
        else:
            confidences = self.probabilities[:,index]
            accuracies = self.acc_matrix[:,index]


        for i, (bin_lower, bin_upper) in enumerate(zip(self.bin_lowers, self.bin_uppers)):
            # Calculated |confidence - accuracy| in each bin
            in_bin = np.greater(confidences,bin_lower.item()) * np.less_equal(confidences,bin_upper.item())
            self.bin_prop[i] = np.mean(in_bin)

            if self.bin_prop[i].item() > 0:
                self.bin_acc[i] = np.mean(accuracies[in_bin])
                self.bin_conf[i] = np.mean(confidences[in_bin])
                self.bin_score[i] = np.abs(self.bin_conf[i] - self.bin_acc[i])

class MaxProbCELoss(CELoss):
    def loss(self, output, labels, n_bins = 15, logits = True):
        self.n_bins = n_bins
        super().compute_bin_boundaries()
        super().get_probabilities(output, labels, logits)
        super().compute_bins()

#http://people.cs.pitt.edu/~milos/research/AAAI_Calibration.pdf
class ECELoss(MaxProbCELoss):

    def loss(self, output, labels, n_bins = 15, logits = True):
        super().loss(output, labels, n_bins, logits)
        return np.dot(self.bin_prop,self.bin_score)

In [56]:
# working out ECE
ansss = []
true_label = []
for hist_id in range(batch_id+1):
    print(hist_id)
    N = len(CIL_cifar_test['y_{}'.format(hist_id)])
    my_id_test = range(len(CIL_cifar_test['y_{}'.format(hist_id)]))
    q = torch.zeros((N, (batch_id + 1) * 10))
    p = torch.zeros(N)
    for i, j in enumerate(my_id_test):
        a = test_model.prediction(test_model, CIL_cifar_test['X_{}'.format(hist_id)][j], 0)
        q[i] = a.cpu()
    ansss += [q * 1.0]
    true_label += [CIL_cifar_test['y_{}'.format(hist_id)][my_id_test].cpu()]
    
pred = torch.vstack(ansss).numpy()
label = torch.concatenate(true_label).numpy()
print(ECELoss().loss(output=pred, labels=label, n_bins = 10, logits = False))

0.3031906019434333


# without functional regularization

In [3]:
# set alignment_reg=0. to turn off alignment regularizer
test_model = CLBruno(x_dim=512, y_dim=128, task_dim=1, cond_dim=129, conv=False, task_num=1,
                     y_cat_num=[10], single_task=True, n_dense_block=6, n_hidden_dense=128,
                     activation=nn.Tanh(), mu_init=0., var_init=1., corr_init=0.1, extractor=True, init_out=10, device=device)
test_model = test_model.to(device)

train_loss, test_loss = test_model.train_init(CIL_cifar_train['X_0'], CIL_cifar_train['y_0'],
                                              torch.zeros(CIL_cifar_train['y_0'].shape[0], dtype=torch.long).to(device),
                                              batch_size=128, epoch=30, weight_decay=0., lr=1e-3, embedding_reg=0.1)
                                              # context_portion=0.2)
batch_sizes = [128]*9
# doing CIL
for batch_id in range(1, 10):
    train_loss1, test_loss1, reg_loss1 = test_model.train_continual_task(X_new=CIL_cifar_train['X_{}'.format(batch_id)],
                                                                         y_new=CIL_cifar_train['y_{}'.format(batch_id)],
                                                                         task_id=0, epoch=30, batch_size=int(batch_sizes[batch_id-1]),
                                                                         weight_decay=0., lr=1e-3, n_pseudo=128,
                                                                         embedding_reg=0.1, alignment_reg=0.)

    acc = 0.
    for hist_id in range(batch_id+1):
        N = len(CIL_cifar_test['y_{}'.format(hist_id)])
        my_id_test = range(len(CIL_cifar_test['y_{}'.format(hist_id)]))
        q = torch.zeros((N, (batch_id + 1) * 10))
        p = torch.zeros(N)
        for i, j in enumerate(my_id_test):
            a, b = test_model.prediction(CIL_cifar_test['X_{}'.format(hist_id)][j], 0)
            q[i] = a.cpu()
            p[i] = b.cpu()
        acc += torch.sum(q.cpu().argmax(1) != CIL_cifar_test['y_{}'.format(hist_id)][my_id_test].cpu()) / N
    print('Batch {}'.format(batch_id))
    print('misclassification: {}'.format(np.round(acc.numpy()/(batch_id+1), 3)))


Batch 1
misclassification: 0.394
Batch 2
misclassification: 0.544
Batch 3
misclassification: 0.606
Batch 4
misclassification: 0.697
Batch 5
misclassification: 0.742
Batch 6
misclassification: 0.783
Batch 7
misclassification: 0.808
Batch 8
misclassification: 0.831
Batch 9
misclassification: 0.854
